In [1]:
library(caret)
library(xgboost)
#library(MlBayesOpt)
library(Matrix)
library(rBayesianOptimization)
library(rjson)
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/304_hyperparameter_optuna.ipynb
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/302_LGBM_BO_hyperpara.ipynb
#https://cran.r-project.org/web/packages/MlBayesOpt/vignettes/MlBayesOpt.html

Loading required package: lattice

Loading required package: ggplot2



In [2]:
set.seed(1)

In [3]:
data_import =function(dataname){
  filename = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/',dataname,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [4]:
data_massage = function(inputs,labels){
    rownamesInput = rownames(inputs)
    inputs        = do.call(data.frame,lapply(inputs, function(x) replace(x, is.infinite(x),NA)))
    naColumns     = colnames(inputs)[colSums(is.na(inputs))>0]
    noVarCol      = getNonVarCols(inputs)
    removeCols    = c(naColumns,noVarCol)
    inputs        = inputs[ , !(colnames(inputs) %in% removeCols)]
    rownames(inputs) = rownamesInput
    labels$min.log.lambda = unlist(lapply(labels$min.log.lambda,exp))
    labels$max.log.lambda = unlist(lapply(labels$max.log.lambda,exp))
    res        = list()
    res$inputs = inputs
    res$labels = labels
    return(res)
}

In [5]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [6]:
getNonVarCols<-function(data){
    var_columns    = apply(inputs,2,var)
    resCol         = names(var_columns[var_columns==0.0])
    return(resCol)
}

In [7]:
# Set Parameters
dataNameRange       = c('ATAC_JV_adipose','CTCF_TDH_ENCODE','H3K27ac-H3K4me3_TDHAM_BP','H3K27ac_TDH_some','H3K36me3_AM_immune')

In [8]:
getaccuracy=function(pred,y_lower,y_higher){
    res = (pred>=y_lower & pred<=y_higher)
    return(res)
}

In [15]:
trainModel = function(X,X_val,y_lower,y_upper,y_lower_val,y_upper_val,param,num_round){
    dtrain = xgb.DMatrix(X)
    setinfo(dtrain,'label_lower_bound', y_lower)
    setinfo(dtrain,'label_upper_bound', y_upper)
    dtest = xgb.DMatrix(X_val)
    setinfo(dtest,'label_lower_bound', y_lower_val)
    setinfo(dtest,'label_upper_bound', y_upper_val)
    watchlist = list(eval = dtest, train = dtrain)
    bst       = xgb.train(param, dtrain, num_round, watchlist,verbose = 0)
    return(bst)
}

In [10]:
res      = data_import(dataNameRange[1])
inputs   = res$inputs
labels   = res$labels
folds    = res$folds
resDataMassage      = data_massage(inputs,labels)
inputs              = resDataMassage$inputs
labels              = resDataMassage$labels
fold_iter           = unique(folds$fold)
accuracy_fold       = numeric(length(fold_iter))

In [44]:
for(i in 1:length(fold_iter)){
    res = getXY(fold_iter[i],folds,inputs,labels)
    X   = res$X
    X.val  = res$X.val
    y.lower= res$y.lower
    y.lower.val = res$y.lower.val
    y.upper     = res$y.upper
    y.upper.val = res$y.upper.val
    json_name   = paste("fold_new",i,"_param.json",sep="")
    result      =  fromJSON(file = json_name)
    eval_metric  = paste("aft-nloglik@",result$distribution,",",result$sigma,sep="")
    num_round    = result$num_round
    
    param = list(eta = result$eta,
                 max_depth = result$max_depth,
                 min_child_weight = result$min_child_weight,
                 subsample = 0.7,
                 alpha = result$reg_alpha,
                 lambda = result$reg_lambda,
                 aft_sigma = result$sigma,
                 aft_noise_distribution=result$distribution,
                 nthread = 4, 
                 verbosity = 0,
                 eval_metric = eval_metric,
                 objective = "aft:survival")
    
    bst= trainModel(X,X,y.lower,y.upper,y.lower,y.upper,param,num_round)
    dtest = xgb.DMatrix(X.val)
    setinfo(dtest,'label_lower_bound', y.lower.val)
    setinfo(dtest,'label_upper_bound', y.upper.val)
    pred.y.val = predict(bst, dtest)
    accuracy_fold[i] = sum(mapply(getaccuracy,pred.y.val,y.lower.val,y.upper.val))/length(pred.y.val)
    
    pred_data           = data.frame(pred.y.val,y.lower.val,y.upper.val)
    colnames(pred_data) = c("predict","y.lower","y.upper")
    accuracy_fold[i]    = getaccuracy(pred.y.val,y.lower.val,y.upper.val)
    fileName            = paste('../../../../result/ATAC_JV_adipose/xgboost/',i,".csv")
    write.table(pred_data,fileName,sep=",",row.names = FALSE)
}

Warning message in accuracy_fold[i] <- getaccuracy(pred.y.val, y.lower.val, y.upper.val):
“number of items to replace is not a multiple of replacement length”
Warning message in accuracy_fold[i] <- getaccuracy(pred.y.val, y.lower.val, y.upper.val):
“number of items to replace is not a multiple of replacement length”
Warning message in accuracy_fold[i] <- getaccuracy(pred.y.val, y.lower.val, y.upper.val):
“number of items to replace is not a multiple of replacement length”
Warning message in accuracy_fold[i] <- getaccuracy(pred.y.val, y.lower.val, y.upper.val):
“number of items to replace is not a multiple of replacement length”


# Save Accuracy

In [45]:
jsonAccuracy = toJSON(accuracy_fold)

In [46]:
write(jsonAccuracy, file="../../../../result/ATAC_JV_adipose/xgboost/accuracy.JSON")